# Convolutional Neural Network

### Importing the libraries

In [40]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
tf.keras.__version__

'3.4.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [42]:
# Prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale= 1./255,         # Feature scaling
    shear_range= 0.2,        
    zoom_range= 0.2,         
    horizontal_flip= True    
)

training_set = train_datagen.flow_from_directory(
    'CNN_dataset/training_set',
    target_size= (64, 64),    # Low image size to put less load on the machine
    batch_size= 32,           # Default value
    class_mode= 'binary'      # Only 2 outcomes (cat/dog), so binary. Use 'categorical' for multi-class classification
)


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [43]:
test_datagen = ImageDataGenerator(rescale=1./255) # applying same feature scalling

test_set = test_datagen.flow_from_directory(
        'CNN_dataset/test_set',
        target_size= (64, 64),
        batch_size= 32,
        class_mode= 'binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [44]:
from tensorflow.keras.models import Sequential

cnn = Sequential()

### Step 1 - Convolution

In [45]:
from tensorflow.keras.layers import Conv2D

cnn.add(Conv2D(
    filters= 32, 
    kernel_size= 3, 
    activation= 'relu', 
    input_shape= [64, 64, 3]
))

# input_shape=[64, 64, 3] assumes RGB images. 
# If the images were in black & white, use input_shape=[64, 64, 1]


c:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [46]:
from tensorflow.keras.layers import MaxPooling2D

MaxPooling2D(
    pool_size= 2,        # Size of the pooling window (height, width)
    strides= 2           # Stride of the pooling operation
)


<MaxPooling2D name=max_pooling2d_6, built=False>

### Adding a second convolutional layer

In [47]:
cnn.add(Conv2D(
    filters=32, 
    kernel_size=3, 
    activation='relu'   #input shape not required as it was fixed in the 1st layer. so it will get the same size as input
))

MaxPooling2D(
    pool_size= 2,
    strides= 2
)

<MaxPooling2D name=max_pooling2d_7, built=False>

### Step 3 - Flattening

In [48]:
from tensorflow.keras.layers import Flatten

cnn.add(Flatten())


### Step 4 - Full Connection

In [49]:
from tensorflow.keras.layers import Dense

cnn.add( Dense(
    units = 128,
    activation= 'relu'
))


### Step 5 - Output Layer

In [50]:
cnn.add( Dense(
    units = 1,                  # binary output so only one layer required
    activation= 'sigmoid'       # sigmoid for output layer
))

## Part 3 - Training the CNN

### Compiling the CNN

In [51]:
cnn.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data= test_set, epochs=25)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('')